# Como calcular estatísticas resumidas

Lembre-se de inicializar o pandas e ler o arquivo titanic.csv

In [1]:
import pandas as pd

titanic = pd.read_csv("../data/titanic.csv")
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Qual é a idade média dos passageiros do Titanic?

Diferentes estatísticas estão disponíveis e podem ser aplicadas a colunas com dados numéricos.  
As operações em geral excluem dados ausentes e operam em todas as linhas por padrão.

In [2]:
titanic["Age"].mean()

np.float64(29.69911764705882)

Qual é a idade média e o preço da passagem dos passageiros do Titanic?

A estatística aplicada a múltiplas colunas do DataFrame (a seleção de duas colunas retorna um DataFrame), é calculada para cada coluna numérica.

In [3]:
titanic[["Age", "Fare"]].median()

Age     28.0000
Fare    14.4542
dtype: float64

A estatística de agregação pode ser calculada para várias colunas ao mesmo tempo.  
Lembra da função describe do primeiro tutorial ?

In [4]:
titanic[["Age", "Fare"]].describe()

,Age,Fare
count,714.000000,891.000000
mean,29.699118,32.204208
std,14.526497,49.693429
min,0.420000,0.000000
25%,20.125000,7.910400
50%,28.000000,14.454200
75%,38.000000,31.000000
max,80.000000,512.329200


Em vez de estatísticas predefinidas, combinações específicas de estatísticas de agregação para colunas,  
podem ser fornecidas e definidas usando o método : DataFrame.agg()

In [5]:
titanic.agg(
    {
        "Age": ["min", "max", "median", "skew"],
        "Fare": ["min", "max", "median", "mean"],
    }
)

,Age,Fare
min,0.420000,0.000000
max,80.000000,512.329200
median,28.000000,14.454200
skew,0.389108,NaN
mean,NaN,32.204208


# Agregando estatísticas agrupadas por categoria

Qual é a idade média dos passageiros homens e mulheres do Titanic?  

Como nosso interesse é a idade média para cada gênero, uma subseleção nestas duas colunas é feita primeiro.  
Em seguida, o método é aplicado à coluna para criar um grupo por categoria.  
A idade média para cada gênero é calculada e retornada.

In [6]:
titanic[["Sex", "Age"]].groupby("Sex").mean()

,Age
Sex,
female,27.915709
male,30.726645


Calcular uma determinada estatística (por exemplo, meanidade) para cada categoria,  
em uma coluna (por exemplo, masculino/feminino na Sexcoluna) é um padrão comum.  
O método groupby é usado para dar suporte a esse tipo de operação.  
Isso se encaixa no split-apply-combinepadrão mais geral:

* Dividir os dados em grupos
* Aplique uma função a cada grupo independentemente
* Combine os resultados em uma estrutura de dados
* As etapas de aplicação e combinação geralmente são feitas juntas no Pandas.

No exemplo anterior, selecionamos explicitamente as duas colunas primeiro.  
Caso contrário, o método mean, é aplicado a cada coluna que contém colunas numéricas, passando numeric_only=True:

In [7]:
titanic.groupby("Sex").mean(numeric_only=True)

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
Sex,,,,,,,
female,431.028662,0.742038,2.159236,27.915709,0.694268,0.649682,44.479818
male,454.147314,0.188908,2.389948,30.726645,0.429809,0.235702,25.523893


Não faz muito sentido obter o valor médio de Pclass.  
Se estivermos interessados ​​apenas na idade média de cada gênero,  
a seleção de colunas (colchetes retangulares, como de costume [])  
também é suportada nos dados agrupados:

In [8]:
titanic.groupby("Sex")["Age"].mean()

Sex
female    27.915709
male      30.726645
Name: Age, dtype: float64

A coluna Pclass, contém dados numéricos, mas, na verdade,  
representa 3 categorias (ou fatores) com os rótulos "1", "2" e "3", respectivamente.  
Calcular estatísticas sobre eles não faz muito sentido.  
Portanto, o Pandas fornece um tipo *Categorical* de dado para lidar com esse tipo de dado.  
Mais informações podem ser encontradas na seção Dados categóricos do guia do usuário .

Qual é o preço médio da passagem para cada uma das combinações de sexo e classe de cabine?

O agrupamento pode ser feito por várias colunas ao mesmo tempo.  
Forneça os nomes das colunas como uma lista para o método groupby().

In [9]:
titanic.groupby(["Sex", "Pclass"])["Fare"].mean()

Sex     Pclass
female  1         106.125798
        2          21.970121
        3          16.118810
male    1          67.226127
        2          19.741782
        3          12.661633
Name: Fare, dtype: float64

Qual é o número de passageiros em cada uma das classes de cabine?

O método value_counts(), conta o número de registros para cada categoria em uma coluna.

In [10]:
titanic["Pclass"].value_counts()

Pclass
3    491
1    216
2    184
Name: count, dtype: int64

A função é um atalho, pois na verdade é uma operação groupby em combinação com a contagem do número de registros dentro de cada grupo:

Ambos sizee countpodem ser usados ​​em combinação com groupby.  
Enquanto sizeincludes NaNvalues ​​e apenas fornece o número de linhas (tamanho da tabela), countexclui os valores ausentes.  
No método value_counts, use o argumento dropna, para incluir ou excluir os valores NaN.

In [11]:
titanic.groupby("Pclass")["Pclass"].count()

Pclass
1    216
2    184
3    491
Name: Pclass, dtype: int64

**LEMBRAR**

* Estatísticas de agregação podem ser calculadas em colunas ou linhas inteiras.
* groupby fornece o poder do padrão dividir-aplicar-combinar .
* value_countsé um atalho conveniente para contar o número de entradas em cada categoria de uma variável.